### Python et BDD

In [ ]:
import sqlite3
import pandas as pd

baseDeDonnees = sqlite3.connect('gtfs_tag.db')
df_agency = pd.read_csv('./SEM-GTFS/agency.txt')
df_agency

In [ ]:
df_agency.to_sql('Agency', con=baseDeDonnees, if_exists='replace', index=False)

In [ ]:
df_stops = pd.read_csv('./SEM-GTFS/stops.txt')
df_stops.head()

In [ ]:
df_stops.to_sql('Stops', con=baseDeDonnees, if_exists='replace', index=False)

***Implémenter des fonctions***

#1. insérer une ligne dans la database

In [ ]:
def getList(dict):
    return dict.keys()

def gen_insert_query(table_name:str, a_dict:dict) -> str:
    '''
    fonction qui génère une commande insertion SQL
    tablename : le nom de la table (e.g. gtfs_stops )
    a_dict : dictionnaire Python
    EXAMPLE OF QUERY : mySql_insert_query = """INSERT INTO Laptop (Id, Name, Price, Purchase_date) 
                                                VALUES 
                                                (15, 'Lenovo ThinkPad P71', 6459, '2019-08-14') """
    '''
    mySql_insert_query = 'INSERT INTO '+table_name+' ('+', '.join(tuple(getList(a_dict)))+')'+' VALUES '+' (:'+', :'.join(tuple(getList(a_dict)))+')'
    return(mySql_insert_query)

In [ ]:
table_name = 'Stops'
a_dict = {'stop_id': 9999,'stop_code': 9999, 'stop_name': '"Ma Maison"', 'stop_lat': 45.2122, 'stop_lon': 5.76728, 'zone_id': None, 'location_type': None, 'parent_station': 'LYON','wheelchair_boarding': 1}
print(gen_insert_query(table_name, a_dict))

In [ ]:
a_dict={'stop_id': 9999,
 'stop_code': 9999,
 'stop_name': '"Ma Maison"',
 'stop_lat': 45.2122,
 'stop_lon': 5.76728,
 'zone_id': None,
 'location_type': None,
 'parent_station': 'LYON',
 'wheelchair_boarding': 1}

In [ ]:
cursor = baseDeDonnees.cursor()
cursor.execute(gen_insert_query(table_name, a_dict), a_dict)
baseDeDonnees.commit()

#2. insérer une liste de lignes dans la database

In [ ]:
def get_insert_queries(tablename:str, df: pd.DataFrame) -> list:
    '''
    fonction qui génère une liste de commande SQL
    tablename : le nom de la table (e.g. gtfs_stops )
    df : le dataframe à insérer
    '''
    list_query = []
    list_dico = df.to_dict(orient='records')
    for minidico in list_dico:
        list_query.append(gen_insert_query(table_name, minidico))
    return(list_query)

In [ ]:
df_toInsert = df_stops.head()
print(get_insert_queries(table_name, df_toInsert))

In [ ]:
cursor = baseDeDonnees.cursor()
i=0
for stringSQL in get_insert_queries(table_name, df_toInsert):
    b_dict = df_toInsert.iloc[i]
    cursor.execute(stringSQL, b_dict)
    baseDeDonnees.commit()
    i+=1

In [ ]:
df_toInsert

#3. implémenter une procédure qui crée un fichier SQL

In [ ]:
def gen_create_file(filename, tablename, df):
    '''
    procédure qui crée un fichier db
    filename : est le nom de fichier (e.g. insert_stops.db)
    tablename : le nom de la relation où nous allons insérer nos tuples (e.g. gtfs_stops)
    df : un dataframe pandas
    '''
    connection = sqlite3.connect(filename)    
    cursor = connection.cursor()
    df.to_sql(tablename, con=connection)

In [ ]:
gen_create_file('newDatabase.db', 'stops', df_toInsert)

In [ ]:
def gen_insert_queries_df(table_name:str, df:pd.DataFrame) -> str:
    dico = df.to_dict(orient='list')
    a_string = f"INSERT INTO {table_name} {tuple(dico.keys())} VALUES {','.join(str(i) for i in tuple(zip(*dico.values())))}"
    a_string = a_string.replace("nan", "'NaN'")
    return a_string

In [ ]:
gen_insert_queries_df(table_name, df_toInsert)

In [ ]:
a_string = gen_insert_queries_df(table_name, df_toInsert)
a_string = a_string.replace("nan", "'NaN'")
a_string

In [ ]:
def gen_insert_file(filename, tablename, df):
    '''
    procédure qui crée un fichier sql
    filename : est le nom de fichier (e.g. insert_stops.sql)
    tablename : le nom de la relation où nous allons insérer nos tuples (e.g. gtfs_stops)
    df : un dataframe pandas
    '''
    with open(filename, 'w') as file:
        file.write(gen_insert_queries_df(table_name, df_toInsert))

In [ ]:
gen_insert_file('file.sql', table_name, df_toInsert)